In [2]:
import pandas as pd
import numpy as np
result = pd.read_csv("../analysis/Household_Survey_Analysis_with_derivations.csv") 
# Constants (from literature)
Y = 0.3  # m³/kg biogas yield per kg of VS, 
VS = 0.8  # Volatile Solids content (fraction of total solids)
retention_time_days = 30  # days

In [3]:
kitchen_waste = result['Solid waste/day (Kg)'].sum()
print(f"Total Solid Waste: {kitchen_waste:.2f} kg/day")

def total_biogas_per_day (kitchen_waste, VS, Y):
    return kitchen_waste * VS * Y

kitchen_biogas_per_day = total_biogas_per_day(kitchen_waste, VS, Y)

#Total biogas yield from sum of daily kitchen waste in households
print(f"\nTotal biogas production potential from kitchen waste: {kitchen_biogas_per_day:.2f} m^3/day")

Total Solid Waste: 36.50 kg/day

Total biogas production potential from kitchen waste: 8.76 m^3/day


In [4]:
#The useful energy required by the households
total_useful_energy = result['Useful Energy per day (MJ/day)'].sum()

print(f"Total Useful Energy Demand: {total_useful_energy:.2f} MJ/day")

# Factors for digester sizing
biogas_energy_content = 20 # MJ/m^3 biogas (from literature)
biogas_stove_efficiency = 0.55 # from literature 
digester_hydraulic_retention_time = 30 # from literature (days)
design_buffer = 1.2 #safety factor 

def required_biogas_yield(total_useful_energy, biogas_energy_content, biogas_stove_efficiency):
    return total_useful_energy / (biogas_energy_content * biogas_stove_efficiency)

Wumba_biogas_need = required_biogas_yield(total_useful_energy, biogas_energy_content, biogas_stove_efficiency)
print(f"\nRequired daily biogas to meet energy demand: {Wumba_biogas_need:.2f} m^3/day")

# --- Incorporating other waste sources ---
# Daily amount and biogas potential of other waste sources in Wumba:
estimated_daily_cow_dung = 50 #kg from estimates of cattle in the area 
biogas_yield_cow_dung = 0.15 #m^3 biogas per kg of cow dung (literature)
cow_dung_biogas = estimated_daily_cow_dung * biogas_yield_cow_dung

estimated_daily_meat_waste = 30 #kg from abattoir estimates
biogas_yield_meat_waste = 0.3 #m^3 biogas per kg of meat waste (literature)
meat_waste_biogas = estimated_daily_meat_waste * biogas_yield_meat_waste

total_waste_biogas_potential = kitchen_biogas_per_day + cow_dung_biogas + meat_waste_biogas
print(f"Total daily biogas potential from all waste sources: {total_waste_biogas_potential:.2f} m^3/day")

#Total Solids (TS%) in the Digester
#Assuming kitchen waste is all solids, cow dung is 25% solids, and meat waste is 30% solids
cow_dung_TS = 0.20 # from literature
meat_waste_TS = 0.25 # from literature
total_solids = (kitchen_waste) + (estimated_daily_cow_dung * cow_dung_TS) + (estimated_daily_meat_waste * meat_waste_TS)
print(f"\nTotal solids in the digester: {total_solids:.2f} kg/day")

#Reactor Slurry Target 
reactor_slurry_target = 0.08 # %target total solids fraction inside the digester slurry, typical for biogas digesters; energypedia
slurry_volume_needed = total_solids / (reactor_slurry_target * 1000 ) #(in m^3/day)
print(f"Slurry Volume: {slurry_volume_needed:.2f} m^3/day")

# Required digester volume based on Hydraulic Retention Time (HRT)
required_digester_volume = slurry_volume_needed * digester_hydraulic_retention_time * design_buffer
print(f"\nBiogas digester volume: {required_digester_volume:.2f} m^3")

# Compare the total biogas potential with the required yield to see if it's sufficient.
print(f"\nIs total waste biogas potential sufficient?")
if total_waste_biogas_potential >= Wumba_biogas_need:
    print("The biogas potential from available waste meets the community's energy needs.")
else:
    print("The biogas potential from available waste does NOT meet the community's energy needs.\n Consider additional waste sources or system optimizations.")

Total Useful Energy Demand: 233.61 MJ/day

Required daily biogas to meet energy demand: 21.24 m^3/day
Total daily biogas potential from all waste sources: 25.26 m^3/day

Total solids in the digester: 54.00 kg/day
Slurry Volume: 0.68 m^3/day

Biogas digester volume: 24.30 m^3

Is total waste biogas potential sufficient?
The biogas potential from available waste meets the community's energy needs.


In [ ]:
#Calculate the digester diameter and height
import math
def digester_dimensions(volume, diameter_to_height_ratio):
    H = (4 * volume / (math.pi * (diameter_to_height_ratio ** 2))) ** (1/3)
    D = diameter_to_height_ratio * H
    return D, H

D, H = digester_dimensions(required_digester_volume, 2)
print(f"Digester Diameter (D): {D:.2f} m")
print(f"Digester Height (H): {H:.2f} m")

Digester Diameter (D): 3.96 m
Digester Height (H): 1.98 m
